**Table of contents**<a id='toc0_'></a>    
- [使用Foursquare数据](#toc1_)    
  - [下载](#toc1_1_)    
    - [参考](#toc1_1_1_)    
    - [AWS S3下载步骤（已验证）](#toc1_1_2_)    
    - [没有验证的方法](#toc1_1_3_)    
      - [方法2: 使用 boto3 脚本并启用多线程](#toc1_1_3_1_)    
      - [方法3: 使用 AWS S3 Transfer Acceleration](#toc1_1_3_2_)    
      - [方法4: 利用 EC2 实例中转](#toc1_1_3_3_)    
      - [最佳实践和优化建议](#toc1_1_3_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[使用Foursquare数据](#toc0_)

## <a id='toc1_1_'></a>[下载](#toc0_)

### <a id='toc1_1_1_'></a>[参考](#toc0_)

1. 信息来源：<https://mp.weixin.qq.com/s/PjvodYVRRKxOflUdeK2c8A>
2. 亚马逊安全凭证管理页面: <https://us-east-1.console.aws.amazon.com/iam/home?region=us-east-1#/security_credentials>
3. 具体数据说明: <https://docs.foursquare.com/data-products/docs/access-fsq-os-places>
    ```log
    Prerequisites
    In order to access Foursquare's Open Source Places data, you will need the following:

    S3 paths:
    - Places - s3://fsq-os-places-us-east-1/release/dt=2024-12-03/places/parquet
    - Categories - s3://fsq-os-places-us-east-1/release/dt=2024-12-03/categories/parquet
    AWS CLI or SDK’s to download the parquet files and use in your local environment
    If using DuckDb, please refer to the Using DuckDb section for instructions on how to create remote/local tables for use.
    ```

### <a id='toc1_1_2_'></a>[AWS S3下载步骤（已验证）](#toc0_)

有ChatGPT提供。

方法1: 使用CLI下载：

1. 注册AWS账号。需要使用信用卡。
2. 下载AWS CLI。下载地址：<https://aws.amazon.com/cn/cli/> 旁边的 "Windows - 下载并运行 64 位 Windows 安装程序。"中进行下载。
3. 在AWS账号管理界面配置用户。**直接使用根用户**（用户权限和用户策略配置非常麻烦）。生成 Access Key ID 和 Secret Access Key 用于数据下载。
4. 使用AWS CLI 中配置用户的Access Key ID 和 Secret Access Key。
   ```cli
   aws configure
   ```
   注意时区设置为数据存储的时区（这好像不必要）。
5. 下载
   1. 下载单个文件
   ```bash
   aws s3 cp s3://fsq-os-places-us-east-1/release/dt=2024-12-03/places/parquet ./local_folder/
   ```
   2. 下载文件夹
   ```bash
   aws s3 cp s3://fsq-os-places-us-east-1/release/dt=2024-12-03/places/parquet ./Foursquare/places/ --recursive
   aws s3 cp s3://fsq-os-places-us-east-1/release/dt=2024-12-03/categories/parquet ./Foursquare/categories/ --recursive
   ```

### <a id='toc1_1_3_'></a>[没有验证的方法](#toc0_)

#### <a id='toc1_1_3_1_'></a>[方法2: 使用 boto3 脚本并启用多线程](#toc0_)

AWS 的 Python SDK boto3 支持高效的数据下载。以下是一个多线程下载的示例代码：

```python
import boto3
import os
from concurrent.futures import ThreadPoolExecutor

# 初始化 S3 客户端
s3 = boto3.client('s3', region_name='us-east-1')

def download_file(bucket_name, key, local_path):
    # 创建本地目录
    os.makedirs(os.path.dirname(local_path), exist_ok=True)
    # 下载文件
    s3.download_file(bucket_name, key, local_path)
    print(f"Downloaded {key} to {local_path}")

def list_s3_objects(bucket_name, prefix):
    # 列出存储桶中的对象
    paginator = s3.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
        for obj in page.get('Contents', []):
            yield obj['Key']

# 下载所有文件
bucket = 'fsq-os-places-us-east-1'
prefix = 'release/dt=2024-12-03/places/parquet/'
local_dir = './local_folder/'

with ThreadPoolExecutor(max_workers=10) as executor:
    for key in list_s3_objects(bucket, prefix):
        local_path = os.path.join(local_dir, key[len(prefix):])
        executor.submit(download_file, bucket, key, local_path)
```

说明：
该脚本使用 ThreadPoolExecutor 实现多线程，提高下载速度。
修改 max_workers 值可以调整并行下载的线程数。


#### <a id='toc1_1_3_2_'></a>[方法3: 使用 AWS S3 Transfer Acceleration](#toc0_)

对于跨区域或全球范围内的高速下载，启用 S3 Transfer Acceleration 会显著提高性能。

启用 Transfer Acceleration：

登录到 S3 控制台。
找到目标存储桶，启用 Transfer Acceleration。
注意：需要管理员权限。
使用 CLI 或 boto3 加速下载：


```bash
aws configure set s3.use_accelerate_endpoint true
aws s3 cp s3://fsq-os-places-us-east-1/release/dt=2024-12-03/places/ ./local_folder/ --recursive
```

#### <a id='toc1_1_3_3_'></a>[方法4: 利用 EC2 实例中转](#toc0_)

如果本地网络速度有限，可以使用 AWS EC2 实例作为中转，下载文件后再传输到本地：

1. 启动 EC2 实例：
   - 在与 S3 存储桶同区域的 AWS 区域中启动 EC2 实例（如 us-east-1）。
2. 配置实例：
    - 确保实例具有正确的 IAM 角色（允许访问目标 S3 存储桶）。
    - 或者在实例中配置 AWS CLI 凭证。
3. 下载数据到 EC2： 使用 AWS CLI 或 boto3 下载数据到实例本地存储。
4. 传输到本地：
   - 使用 SCP 或其他工具将数据从 EC2 下载到本地：
    ```bash
    scp -i your-key.pem ec2-user@your-ec2-ip:/path/to/data ./local_folder/
    ```

#### <a id='toc1_1_3_4_'></a>[最佳实践和优化建议](#toc0_)

1. 优化区域： 确保您所在的区域和存储桶区域一致（us-east-1），以减少网络延迟。
2. 并发下载：
   - 使用 CLI 的并发选项：--parallel（需要 AWS CLI v2 支持）。
   - 或者在自定义脚本中实现多线程或多进程。
3. 减少冗余请求：
   - 使用 --only-show-errors 减少日志。
   - 对于大批量文件，启用 S3 分段下载（boto3 自动支持）。
4. 监控下载性能： 使用 --debug 检查下载性能瓶颈。

## 使用